In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import torchvision.io as io
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, Subset

import torch
import torch.nn as nn
import torch.nn.functional as F 

from transformers import AutoImageProcessor, AutoModelForImageClassification
from torch.optim import Adam
from tqdm import tqdm

device='cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
#get a list of all the files of the dataset
casia_path=Path("/kaggle/input/casiadataset/imgs/imgs")
casia_files=casia_path.glob("*/*.jpg")
casia_file_list=list(casia_files)

In [ ]:
# a function that takes a path and returns the image tensor/kaggle/input/casiadataset/imgs
def read_image(path):
    image_tensor = io.read_image(path)
    return image_tensor

In [ ]:
casia_df=pd.DataFrame(
    data={
        "subject": [str(file).split("/")[-2] for file in casia_file_list],
        "file_name": [str(file).split("/")[-1] for file in casia_file_list],
        "file_path": [str(file) for file in casia_file_list],
        # 'tensor': [load_image(str(file)) for file in tqdm(casia_file_list)]
    }
)
casia_df['subject'] = casia_df['subject'].astype(int)
print(casia_df)

In [ ]:
casia_df['split']='train'
test_indices = casia_df[casia_df['split']=='train'].groupby(by='subject').sample(frac=0.1).index #prendre 10% par subject
casia_df.loc[test_indices,'split']='test'
val_indices = casia_df[casia_df['split']=='train'].groupby(by='subject').sample(frac=0.1).index #prendre 10% par subject
casia_df.loc[val_indices,'split']='val'
casia_df

In [ ]:
class CasiaDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe=dataframe
    def __len__(self):
        return len(self.dataframe)
    def __getitem__(self,idx): #this function returns a pair of images

        image_tensor_1=read_image(self.dataframe.iloc[idx]['file_path']) #we read the first image
        subject_1=self.dataframe.iloc[idx]['subject'] #we get the subject of the image (the person)

        return image_tensor_1, subject_1
    

In [ ]:
#create the object dataset
casia_train=CasiaDataset(casia_df[casia_df['split']=='train'])
casia_val=CasiaDataset(casia_df[casia_df['split']=='val'])
casia_test=CasiaDataset(casia_df[casia_df['split']=='test'])

In [ ]:
image_tensor_1, subject_1 = casia_train[500]
plt.imshow(image_tensor_1.numpy().transpose(1,2,0))
plt.show()

In [ ]:
casia_dataloader_train = DataLoader(casia_train, batch_size=128, shuffle=True, drop_last=True)
casia_dataloader_val = DataLoader(casia_val, batch_size=128, shuffle=False, drop_last=True)
casia_dataloader_test = DataLoader(casia_test, batch_size=128, shuffle=False, drop_last=True)

In [ ]:
image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-18") #pour traiter l'image
model = AutoModelForImageClassification.from_pretrained("microsoft/resnet-18") #mon modèle

In [ ]:
nb_classes = casia_df['subject'].unique().shape[0]
print(nb_classes)

In [ ]:
#modifier la derniere couche du modèle pour avoir 10572 classes
model.classifier = nn.Sequential(
    nn.Flatten(start_dim=1, end_dim=-1),
    nn.Linear(in_features=512, out_features=nb_classes, bias=True)             
)

In [ ]:
def accuracy_fn(logits, labels):
    y_probs=F.softmax(logits, dim=-1)
    y_preds=y_probs.argmax(dim=-1)
    accuracy=(y_preds==labels).sum()/len(labels)
    return accuracy

In [ ]:
model.to(device)
#boucle d'entrainement
optimizer = Adam(params = model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss().to(device)

nb_epochs=100
accuracies_g=[]
train_losses_g=[]
val_losses_g=[]
best_validation=1e9

for epoch in range(nb_epochs):
    accuracies=[]
    train_losses=[]
    val_losses=[]
    
    model.train()
    for batch, labels in tqdm(casia_dataloader_train):
        batch=batch.to(device)
        labels=labels.to(device)
        optimizer.zero_grad() #mettre à zero les gradients
        processed_batch=image_processor(batch, return_tensors="pt")
        logits = model(processed_batch['pixel_values'].to(device))['logits']
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    
    model.eval()
    with torch.no_grad():
        for batch, labels in tqdm(casia_dataloader_val):
            batch=batch.to(device)
            labels=labels.to(device)
            processed_batch=image_processor(batch, return_tensors="pt")
            logits = model(processed_batch['pixel_values'].to(device))['logits']
            loss=loss_fn(logits, labels)
            accuracy=accuracy_fn(logits, labels)
            accuracies.append(accuracy.item())
            val_losses.append(loss.item())
            
    accuracies_g.append(torch.tensor(accuracies).mean().item())
    train_losses_g.append(torch.tensor(train_losses).mean().item())
    val_losses_g.append(torch.tensor(val_losses).mean().item())
    print(f"Epoch: {epoch}, train_loss: {train_losses_g[-1]}, val_loss: {val_losses_g[-1]}, accuracy: {accuracies_g[-1]}")
    if best_validation>val_losses_g[-1]:
        torch.save(model.state_dict(), '/kaggle/working/model.pt')
        best_validation=val_losses_g[-1]
    #sil ya cinq loss sans amélioration de la validation, on arrete
    if len(val_losses_g)>4 and (np.array(val_losses_g[-3:]).min() > train_losses_g[-4]):
        break
    torch.save({'val_loss': val_losses_g,
               'train_loss': train_losses_g,
               'accuracy': accuracies_g}, '/kaggle/working/train_stats.pt')


In [ ]:
# loaded_metrics = torch.load('/kaggle/working/train_stats.pt')
# loaded_metrics